In [ ]:
%pip install psycopg2-binary
%pip install python-dotenv
%pip install soundfile

In [ ]:
import os
from typing import Any
import psycopg2
from psycopg2.extras import DictCursor
from dotenv import load_dotenv
load_dotenv()


def get_connection():
    host = os.environ.get("PGHOST")
    dbname = os.environ.get("PGDATABASE")
    user = os.environ.get("PGUSER")
    password = os.environ.get("PGPASSWORD")
    return psycopg2.connect(host=host, dbname=dbname, user=user, password=password)

def part_name_to_id(part_name: str):
    response = None
    with get_connection() as conn:
        with conn.cursor(cursor_factory=DictCursor) as cur:
            cur.execute("SELECT id FROM parts where name=%s", (part_name,))
            result = cur.fetchone()
            response = dict(result)["id"] if result is not None else None

    return response



def insert_loop(name: str, part_id: int, excitement: int, data: bytes) -> list[dict[str, Any]]:
  sql = """
  insert into loops (name, part_id, excitement, data) values (%s, %s, %s, %s);
  """
  response = None
  with get_connection() as conn:
    with conn.cursor(cursor_factory=DictCursor) as cur:
      cur.execute(sql, (name, part_id, excitement, data))

def get_loop():
  sql = """
  select * from loops limit 1;
  """
  response = None
  with get_connection() as conn:
    with conn.cursor(cursor_factory=DictCursor) as cur:
      cur.execute(sql)
      result = cur.fetchone()
      response = dict(result) if result is not None else None

  return response

print(part_name_to_id("Bass"))

In [ ]:
import IPython
import glob
import os
import soundfile as sf
import IPython.display as ipd

base_dir = os.environ.get("LOOPS_DRIVE_DIR")
loops_dir = os.path.join(base_dir, "Musicloops")
part_dirs = glob.glob(os.path.join(loops_dir, "*"))



def folder2part(folder_name: str) -> str:
  folder_names = ["Basses", "Drums", "Synth", "Sequences"]
  parts = ["Bass", "Drums", "Synth", "Sequence"]
  f2p = {f: p for f,p in zip(folder_names, parts)}
  return f2p[folder_name]


def insert_loop_data():
  for part_dir in part_dirs:
    excitement_dirs = glob.glob(os.path.join(part_dir, "*"))
    dir_name = os.path.basename(part_dir)
    part_name = folder2part(dir_name)
    part_id = int(part_name_to_id(part_name))

    for excitement_dir in excitement_dirs:
      excitement_value = int(os.path.basename(excitement_dir))
      loop_files = glob.glob(os.path.join(excitement_dir, "*"))

      for loop_file in loop_files:
        loop_name = os.path.splitext(os.path.basename(loop_file))[0]
        data = None
        with open(loop_file, mode='rb') as wf:
          data = wf.read()

        insert_loop(loop_name, part_id, excitement_value, data)

# insert_loop_data()

In [ ]:
loop = get_loop()

In [ ]:
print(type(loop["data"].tobytes()))
ipd.Audio(data=loop["data"].tobytes())